# Products

Our goal is to find product **categories** that repeatdely underperform vs others, and understand why

## 0 - `product.py` 

We have given you the solution to `product.py` in your challenge folder
👉 Copy-paste it to your local olist/product.py folder

It provides aggregates at a `product_id` level of the various orders that have taken place with Olist

`get_training_data` method in `olist/product.py` returns the following DataFrame:

  - `product_id` (_str_) _the id of the product_ **UNIQUE**
  - `category` (_str_) _the category name (in English)_
  - `product_name_length` (_float_) _character length of product name_
  - `product_description_length` (_float_) _character length of product description_
  - `product_photos_qty` (_int_) _the number of photos for the product_
  - `product_weight_g` (_float_) _weight of the product (in g)_
  - `product_length_cm` (_float_) _length of the product (in cm)_
  - `product_height_cm` (_float_) _height of the product (in cm)_
  - `product_width_cm` (_float_) _width of the product (in cm)_
  - `price` (_float_) _average price at which the product is sold_
  - `wait_time` (_float_) _the average wait time in days for orders in which the product was sold._
  - `share_of_five_stars` (_float_) _the share of five star orders for orders in which the product was sold_
  - `share_of_one_stars` (_float_) _the share of one star orders for orders in which the product was sold_
  - `review_score` (_float_) _the average review score of the order in which each product is sold_
  - `n_orders` (_int_) _the number of orders in which the product appeared_
  - `quantity` (_int_) _the total number of products sold_
  - `sales` (_float_) _the total value of sales in $BRL for the product_

## 1 - Analysis per product_id

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

❓ Inspect the new `Product().get_training_data()` dataframe, for instance by plotting histograms of each variable using `plt.hist()`. 

In [4]:
from olist.product import Product
products = Product().get_training_data()

In [3]:
products.head()

In [4]:
products.describe()

In [5]:
products.hist()
fig = plt.gcf()
fig.set_size_inches(15,10)

In [9]:
plt.figure(figsize=(20,20))
for (i, col) in enumerate(products.set_index('product_id').drop(columns='category').columns):
    plt.subplot(5,5,i+1)
    sns.histplot(products[col])

❓ Model review_score by an OLS with the continuous feature of your choice, and discover the R-squared and important features

## 2 - Aggregation per product categories

### 2.1 - Build aggregated dataframe

❓ Create a function `get_product_cat` which accepts an aggregating method as an argument and returns a DataFrame with each `product_category`'s `quantity` summed and all other non `str` type properties aggregated with the passed method.  
For instance `get_product_cat('median')` returns:

  - `quantity` (sum)
  - `wait_time` (median)
  - `review_score` (median)
  - `price` (median)
  - ....

In [7]:
products.category.nunique()

In [10]:
products.groupby("category")['product_id'].count()

In [11]:
sns.displot(products.groupby("category")['product_id'].count())
plt.title('number of unique product_id per category')

In [5]:
def get_product_cat(agg="mean"):
    columns = list(products.select_dtypes(exclude=['object']).columns)
    agg_params = dict(zip(columns, [agg] * len(columns)))
    agg_params['quantity'] = 'sum'

    return products.groupby("category").agg(agg_params)


### Test code

In [7]:
from nbresult import ChallengeResult
product_cat = get_product_cat('mean')
result = ChallengeResult('products',
    shape=product_cat.shape,
    avg_review_score=int(product_cat['review_score'].mean()),
    avg_price=int(product_cat['price'].mean()),
    avg_quantity=int(product_cat['quantity'].mean())
)
result.write()
print(result.check())

### 2.2 - Exploration

❓ What are the best performing product categories?

In [128]:
product_cat[['review_score']].sort_values(by='review_score')

❓ Let's try to understand _why_ some categories are performing better than others. 

Using plotly, create different scatterplots, varying `x`, `y`, `color` and `size`, to finds clues about factors impacting the "review_score". 

- Do you notice underperforming product categories?
- Can you think of a strategy to improve Olist's profit margin as per the CEO request?

<details>
    <summary>Hints</summary>

Try plotting `product_length_cm` against `wait_time`, with color as `review_score`, and bubble size as "sales" for instance
</details>

In [112]:
# YOUR CODE BELOW
import plotly.express as px

In [69]:
product_cat['sales'] = product_cat['price'] * product_cat['quantity']

In [113]:
fig = px.scatter(
    product_cat[product_cat['quantity'] > 200].reset_index(),
    x="wait_time",
    y="product_length_cm",
    size="sales",
    hover_name="category",
    color="review_score",
    size_max=40,
)
fig.show()

### 2.3 - Causal inference

☝️ It seems that large products like furniture, which happen to take longer to deliver, are performing worse than other products. Are consumers disappointed about the product itself, or by the slow delivery time?

❓ Run an OLS to model `review_score` so as to isolate the true contribution of each product category on customer satisfaction, by holding `wait_time` constant? 

- Which dataset should you use for this regression? `product_cat` or the entire `products` training dataset?

- Which regressors / independent variables / features should you use? 

Investigate the results: which product categories correlate with higher review_score holding wait_time constant?

Feel free to use `return_significative_coef(model)` coded for you in `olist/utils.py`

In [122]:
model = smf.ols(formula='review_score ~ C(category) + wait_time', data=data).fit()

In [123]:
print(model.rsquared)

In [124]:
model.summary()

In [130]:
# top 5 coef with p-value < 0.05
from olist.utils import return_significative_coef
return_significative_coef(model)

☝️ Furniture is no longer in the list of signigicant coefficients. The problem may have come from delivery rather than the product itself! On the contrary, books are regularly driving higher reviews, even after accounting for generally quicker delivery time. 

🏁 **Congratulations with this final challenge! Don't forget to commit and push your analysis**